In [31]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import pulp

In [32]:
drivers = {
    "NOR" : [59, 41, 27, 32, 36],
    "VER" : [29, 30, 26, 22, 38],
    "ANT" : [32, 29, 23, 14, 15],
    "PIA" : [10, 45, 24, 45, 38],
    "RUS" : [25, 35, 18, 31, 16],
    "ALB" : [17, 11, 8, 15, 10],
    "STR" : [16, 17, 2, 7, 1],
    "OCO" : [8, 24, 1, 15, 6],
    "BEA" : [2, 20, 4, 22, 5],
    "HUL" : [20, 2, 1, -20, 3],
    "LEC" : [12, -12, 20, 20, 25],
    "HAM" : [4, -1, 11, 38, 14],
    "TSU" : [None, None, 13, 9, -17],
    "HAD" : [-20, 11, 8, 5, 8],
    "DOO" : [-20, 11, 6, 2, 1],
    "LAW" : [None, None, -4, 11, 5],
    "SAI" : [-19, 8, 4, -9, 12],
    "GAS" : [1, -8, 0, 13, -17],
    "BOR" : [-18, 7, -2, 4, 3],
    "ALO" : [-20, -16, 4, 5, 7]
}

teams = {
    "mclaren" : [71, 101, 71, 92, 86],
    "mercedes" : [67, 69, 56, 60, 46],
    "redbull" : [19, 67, 46, 46, 21],
    "haas" : [14, 47, 10, 40, 12],
    "ferrari" : [36, -23, 56, 73, 64],
    "williams" : [10, 26, 19, 13, 27],
    "sauber" : [3, 12, 0, -22, 10],
    "aston" : [1, 4, 9, 18, 11],
    "alpine" : [-14, -6, 9, 22, -6],
    "rb" : [-10, 42, 14, 22, 18]
}

prices = {
    "NOR": 30.1,   "VER": 28.9,    "ANT": 19.6,    "PIA": 23.7,     "RUS": 22.1,
    "ALB": 12.6,   "STR": 9.1,     "OCO": 9.1,     "BEA": 8.5,      "HUL": 6.6,
    "LEC": 24.6,   "HAM": 23.3,    "TSU": 15.8,    "HAD": 5.7,      "DOO": 5,
    "LAW": 6.6,    "GAS": 8.8,    "SAI": 10.1,    "BOR": 4.5,      "ALO": 6.2,
    "mclaren": 31.5, "mercedes": 24.2, "redbull": 26.3, "haas": 10,
    "ferrari": 27.6, "williams": 15.3, "rb": 9.8,      "sauber": 4.8,
    "aston": 8.3,    "alpine": 8.1
}


In [33]:
great = 1.2
good = 0.9
bad = 0.6

In [34]:
points = {}
for name, scores in drivers.items():
    points[name] = scores
for name, scores in teams.items():
    points[name] = scores
points

{'NOR': [59, 41, 27, 32, 36],
 'VER': [29, 30, 26, 22, 38],
 'ANT': [32, 29, 23, 14, 15],
 'PIA': [10, 45, 24, 45, 38],
 'RUS': [25, 35, 18, 31, 16],
 'ALB': [17, 11, 8, 15, 10],
 'STR': [16, 17, 2, 7, 1],
 'OCO': [8, 24, 1, 15, 6],
 'BEA': [2, 20, 4, 22, 5],
 'HUL': [20, 2, 1, -20, 3],
 'LEC': [12, -12, 20, 20, 25],
 'HAM': [4, -1, 11, 38, 14],
 'TSU': [None, None, 13, 9, -17],
 'HAD': [-20, 11, 8, 5, 8],
 'DOO': [-20, 11, 6, 2, 1],
 'LAW': [None, None, -4, 11, 5],
 'SAI': [-19, 8, 4, -9, 12],
 'GAS': [1, -8, 0, 13, -17],
 'BOR': [-18, 7, -2, 4, 3],
 'ALO': [-20, -16, 4, 5, 7],
 'mclaren': [71, 101, 71, 92, 86],
 'mercedes': [67, 69, 56, 60, 46],
 'redbull': [19, 67, 46, 46, 21],
 'haas': [14, 47, 10, 40, 12],
 'ferrari': [36, -23, 56, 73, 64],
 'williams': [10, 26, 19, 13, 27],
 'sauber': [3, 12, 0, -22, 10],
 'aston': [1, 4, 9, 18, 11],
 'alpine': [-14, -6, 9, 22, -6],
 'rb': [-10, 42, 14, 22, 18]}

In [35]:
points = pd.DataFrame.from_dict(points).set_index(pd.Index([1,2,3,4,5]))

points = points.apply(pd.to_numeric, errors='coerce')

Validating thresholds

In [36]:
alpha = 0.5

ewm = points.ewm(alpha=alpha, adjust=False).mean().iloc[-1]
ewm.name = "E"

points = pd.concat([points, ewm.to_frame().T], axis=0)
points

,NOR,VER,ANT,PIA,RUS,ALB,STR,OCO,BEA,HUL,...,mclaren,mercedes,redbull,haas,ferrari,williams,sauber,aston,alpine,rb
1,59.000,29.0000,32.0000,10.0000,25.00,17.0,16.0000,8.000,2.000,20.0,...,71.000,67.0,19.000,14.0000,36.0000,10.000,3.0000,1.0000,-14.000,-10.00
2,41.000,30.0000,29.0000,45.0000,35.00,11.0,17.0000,24.000,20.000,2.0,...,101.000,69.0,67.000,47.0000,-23.0000,26.000,12.0000,4.0000,-6.000,42.00
3,27.000,26.0000,23.0000,24.0000,18.00,8.0,2.0000,1.000,4.000,1.0,...,71.000,56.0,46.000,10.0000,56.0000,19.000,0.0000,9.0000,9.000,14.00
4,32.000,22.0000,14.0000,45.0000,31.00,15.0,7.0000,15.000,22.000,-20.0,...,92.000,60.0,46.000,40.0000,73.0000,13.000,-22.0000,18.0000,22.000,22.00
5,36.000,38.0000,15.0000,38.0000,16.00,10.0,1.0000,6.000,5.000,3.0,...,86.000,46.0,21.000,12.0000,64.0000,27.000,10.0000,11.0000,-6.000,18.00
E,35.625,31.4375,17.6875,36.6875,21.75,11.5,4.5625,8.875,9.875,-2.0,...,85.625,53.5,33.125,21.0625,58.0625,21.375,0.4375,11.4375,2.375,18.25


In [37]:
s = points.iloc[-3:-1].sum()

rows = []
for idx, val in s.items():
    row = {
        "Driver/Team" : idx,
        "Terrible/Bad" : round(3 * bad * prices[idx] - val, 2),
        "Bad/Good" : round(3 * good * prices[idx] - val, 2),
        "Good/Great"  : round(3 * great * prices[idx] - val, 2),
        "Expected" : points.loc['E', idx]
    }
    rows.append(row)
    
thresholds = pd.DataFrame(rows).set_index('Driver/Team')
thresholds


,Terrible/Bad,Bad/Good,Good/Great,Expected
Driver/Team,,,,
NOR,-13.82,13.27,40.36,35.6250
VER,-7.98,18.03,44.04,31.4375
ANT,6.28,23.92,41.56,17.6875
PIA,-40.34,-19.01,2.32,36.6875
RUS,-7.22,12.67,32.56,21.7500
ALB,-2.32,9.02,20.36,11.5000
STR,8.38,16.57,24.76,4.5625
OCO,-4.62,3.57,11.76,8.8750
BEA,-11.70,-4.05,3.60,9.8750


In [38]:
rows = []

for x in points.columns:
    mean = points.loc['E', x]
    std = points[x].std()

    z = (thresholds.loc[x, 'Good/Great'] - mean) / std
    p_great = round(1 - norm.cdf(z), 2)

    z = (thresholds.loc[x, 'Bad/Good'] - mean) / std
    p_good = round(1 - norm.cdf(z) - p_great, 2)

    z = (thresholds.loc[x, 'Terrible/Bad'] - mean) / std
    p_bad = round(1 - norm.cdf(z) - p_great - p_good, 2)

    p_terrible = round(norm.cdf(z), 2)

    rows.append({"Driver/Team": x, "P_Terrible": p_terrible, "P_Bad": p_bad, "P_Good": p_good, "P_Great": p_great})
    
probs = pd.DataFrame(rows).set_index('Driver/Team')
probs

,P_Terrible,P_Bad,P_Good,P_Great
Driver/Team,,,,
NOR,0.00,0.02,0.65,0.33
VER,0.00,0.01,0.98,0.01
ANT,0.06,0.74,0.20,0.00
PIA,0.00,-0.00,0.01,0.99
RUS,0.00,0.11,0.82,0.07
ALB,0.00,0.23,0.77,0.00
STR,0.71,0.25,0.04,0.00
OCO,0.05,0.20,0.39,0.36
BEA,0.01,0.04,0.18,0.77


In [39]:
rows_driver = []
rows_team = []

for x in points.columns:
    expected_points = points.loc['E', x]

    std = points[x].std()

    lower_bound = round(expected_points - norm.cdf(0.975) * std, 2)
    upper_bound = round(expected_points + norm.cdf(0.975) * std, 2)

    if prices[x] < 5.1:
        diff = prices[x] - 4.5
        expected_price_change = 0.6*probs.loc[x, 'P_Great'] + 0.2*probs.loc[x, 'P_Good'] - min(0.2, diff) *probs.loc[x, 'P_Bad'] - min(0.6, diff) *probs.loc[x, 'P_Terrible']
    else:
        expected_price_change = 0.3*probs.loc[x, 'P_Great'] + 0.1*probs.loc[x, 'P_Good'] - 0.1*probs.loc[x, 'P_Bad'] - 0.3*probs.loc[x, 'P_Terrible']
        if prices[x] < 19:
            expected_price_change *= 2
    
    expected_price_change = round(expected_price_change, 2)


    if x in drivers.keys():
        rows_driver.append({"Driver": x, "LowerBound": lower_bound, "UpperBound": upper_bound, "------     Price": prices[x], "E[Points]": expected_points, "E[PriceChange]": expected_price_change})
    else:
        rows_team.append({"Team": x, "LowerBound": lower_bound, "UpperBound": upper_bound, "------     Price": prices[x], "E[Points]": expected_points, "E[PriceChange]": expected_price_change})
    
team = pd.DataFrame(rows_team).set_index('Team')
driver = pd.DataFrame(rows_driver).set_index('Driver')
driver

,LowerBound,UpperBound,------ Price,E[Points],E[PriceChange]
Driver,,,,,
NOR,26.36,44.89,30.1,35.6250,0.16
VER,26.94,35.93,28.9,31.4375,0.10
ANT,11.42,23.95,19.6,17.6875,-0.07
PIA,25.26,48.12,23.7,36.6875,0.30
RUS,15.56,27.94,22.1,21.7500,0.09
ALB,8.72,14.28,12.6,11.5000,0.11
STR,-1.26,10.38,9.1,4.5625,-0.47
OCO,2.17,15.58,9.1,8.8750,0.22
BEA,2.71,17.04,8.5,9.8750,0.48


In [40]:
team

,LowerBound,UpperBound,------ Price,E[Points],E[PriceChange]
Team,,,,,
mclaren,75.77,95.48,31.5,85.6250,0.30
mercedes,46.29,60.71,24.2,53.5000,0.30
redbull,18.00,48.25,26.3,33.1250,0.21
haas,7.94,34.18,10.0,21.0625,0.60
ferrari,28.80,87.32,27.6,58.0625,0.30
williams,15.65,27.10,15.3,21.3750,0.53
sauber,-9.69,10.57,4.8,0.4375,-0.28
aston,6.43,16.45,8.3,11.4375,0.58
alpine,-8.38,13.13,8.1,2.3750,-0.12


In [41]:
def optimize_team(prev_drivers, prev_teams, extra_money, free_transfers, dollar_to_point):
    # === Initial Budget ===
    budget = sum(prices[d] for d in prev_drivers) + sum(prices[t] for t in prev_teams) + extra_money
    print("Budget:", budget)

    transfer_penalty = 10
    price_weight = dollar_to_point

    # === LP Problem ===
    prob = pulp.LpProblem("Maximize", pulp.LpMaximize)

    # === Decision Variables ===
    x_driver = {name: pulp.LpVariable(f"x_p_{name}", cat="Binary") for name in driver.index}
    x_team   = {name: pulp.LpVariable(f"x_t_{name}", cat="Binary") for name in team.index}
    captain  = {name: pulp.LpVariable(f"c_{name}", cat="Binary") for name in driver.index}

    # === Transfer flags and slack
    t_driver = {name: pulp.LpVariable(f"t_p_{name}", cat="Binary") for name in driver.index}
    t_team   = {name: pulp.LpVariable(f"t_t_{name}", cat="Binary") for name in team.index}
    extra_transfers = pulp.LpVariable("extra_transfers", lowBound=0, cat="Integer")

    # === Constraints ===
    prob += pulp.lpSum(x_driver.values()) == 5, "NumDrivers"
    prob += pulp.lpSum(x_team.values()) == 2, "NumTeams"
    prob += pulp.lpSum(captain.values()) == 1, "OneCaptain"

    # Captain must be a selected driver
    for name in driver.index:
        prob += captain[name] <= x_driver[name], f"CaptainOnlyIfSelected_{name}"

    # Budget
    prob += (
        pulp.lpSum(prices[name] * x_driver[name] for name in driver.index) +
        pulp.lpSum(prices[name] * x_team[name]   for name in team.index)
    ) <= budget, "Budget"

    # Transfers
    for name in driver.index:
        if name in prev_drivers:
            prob += t_driver[name] >= 1 - x_driver[name]
        else:
            prob += t_driver[name] >= x_driver[name]
    for name in team.index:
        if name in prev_teams:
            prob += t_team[name] >= 1 - x_team[name]
        else:
            prob += t_team[name] >= x_team[name]
    prob += (
        (pulp.lpSum(t_driver.values()) + pulp.lpSum(t_team.values())) / 2
    ) <= extra_transfers + free_transfers, "TransferSlack"

    # === Objective Function ===
    point_term = (
        pulp.lpSum(driver.loc[name, "E[Points]"] * (x_driver[name] + captain[name]) for name in driver.index) +
        pulp.lpSum(team.loc[name, "E[Points]"] * x_team[name] for name in team.index)
    )
    price_term = (
        pulp.lpSum(driver.loc[name, "E[PriceChange]"] * x_driver[name] for name in driver.index) +
        pulp.lpSum(team.loc[name, "E[PriceChange]"] * x_team[name] for name in team.index)
    )

    prob += point_term + price_weight * price_term - transfer_penalty * extra_transfers, "TotalObjective"

    # === Solve ===
    prob.solve()

    # === Output Selection ===
    selected_players = {n for n in driver.index if pulp.value(x_driver[n]) == 1}
    selected_teams   = {n for n in team.index   if pulp.value(x_team[n])   == 1}
    captain_selected = [n for n in driver.index if pulp.value(captain[n]) == 1]
    captain_selected = captain_selected[0] if captain_selected else None

    # Transfers
    prev_players = set(prev_drivers)
    prev_teamset = set(prev_teams)

    total_transfers = (
        len(selected_players.symmetric_difference(prev_players)) +
        len(selected_teams.symmetric_difference(prev_teamset))
    ) // 2
    extra = max(0, total_transfers - free_transfers)
    penalty_applied = extra * transfer_penalty

    # Points and price change breakdown
    total_points = sum(driver.loc[n, "E[Points]"] * pulp.value(x_driver[n]) for n in driver.index) \
                 + sum(team.loc[n, "E[Points]"]   * pulp.value(x_team[n])   for n in team.index) \
                 + (driver.loc[captain_selected, "E[Points]"] if captain_selected else 0)

    total_price_change = (
        sum(driver.loc[n, "E[PriceChange]"] * pulp.value(x_driver[n]) for n in driver.index) +
        sum(team.loc[n, "E[PriceChange]"]   * pulp.value(x_team[n])   for n in team.index)
    )

    # === Final Output ===
    print("Selected Players:", sorted(selected_players))
    print("Selected Teams:  ", sorted(selected_teams))
    print("Double Points:         ", captain_selected)

    print(f"\nExpected Points:       {total_points:.2f}")
    print(f"Expected Price Change: {total_price_change:.2f}")
    print(f"Weighted Objective:    {pulp.value(prob.objective):.2f}")
    print(f"Total Transfers:       {total_transfers}")
    print(f"Extra Transfers:       {extra}")
    print(f"Penalty Applied:       {penalty_applied}")

    sel_drivers_df = driver.loc[list(selected_players)].assign(
        Price=lambda df: df.index.map(prices),
        Captain=lambda df: df.index == captain_selected
    )
    sel_teams_df = team.loc[list(selected_teams)].assign(Price=lambda df: df.index.map(prices))

    print("\nDrivers selected:")
    print(sel_drivers_df.iloc[:, 2:])
    print("\nTeams selected:")
    print(sel_teams_df.iloc[:, 2:])


In [42]:
dollar_to_points = 10

In [43]:
optimize_team(
    prev_drivers=["PIA", "ANT", "BEA", "HAD", "RUS"],
    prev_teams=["haas", "rb"],
    extra_money=4.3,
    free_transfers=2,
    dollar_to_point=dollar_to_points
)


Budget: 103.7
Selected Players: ['ANT', 'BEA', 'BOR', 'HAD', 'PIA']
Selected Teams:   ['haas', 'mclaren']
Double Points:          PIA

Expected Points:       214.88
Expected Price Change: 1.89
Weighted Objective:    233.77
Total Transfers:       2
Extra Transfers:       0
Penalty Applied:       0

Drivers selected:
        ------     Price  E[Points]  E[PriceChange]  Price  Captain
Driver                                                             
BEA                  8.5     9.8750            0.48    8.5    False
HAD                  5.7     5.6875            0.13    5.7    False
BOR                  4.5     1.5625            0.15    4.5    False
PIA                 23.7    36.6875            0.30   23.7     True
ANT                 19.6    17.6875           -0.07   19.6    False

Teams selected:
         ------     Price  E[Points]  E[PriceChange]  Price
Team                                                       
haas                 10.0    21.0625             0.6   10.0
mclaren   

In [44]:
optimize_team(
    prev_drivers=["PIA", "BEA", "ANT", "BOR", "HAD"],
    prev_teams=["mclaren", "rb"],
    extra_money=0,
    free_transfers=2,
    dollar_to_point=dollar_to_points
    
)

Budget: 103.3
Selected Players: ['BEA', 'BOR', 'DOO', 'HAD', 'PIA']
Selected Teams:   ['mclaren', 'mercedes']
Double Points:          PIA

Expected Points:       230.81
Expected Price Change: 1.37
Weighted Objective:    244.51
Total Transfers:       2
Extra Transfers:       0
Penalty Applied:       0

Drivers selected:
        ------     Price  E[Points]  E[PriceChange]  Price  Captain
Driver                                                             
BEA                  8.5     9.8750            0.48    8.5    False
HAD                  5.7     5.6875            0.13    5.7    False
BOR                  4.5     1.5625            0.15    4.5    False
PIA                 23.7    36.6875            0.30   23.7     True
DOO                  5.0     1.1875           -0.29    5.0    False

Teams selected:
          ------     Price  E[Points]  E[PriceChange]  Price
Team                                                        
mercedes              24.2     53.500             0.3   24.2
mcl

In [45]:
optimize_team(
    prev_drivers=["PIA", "BEA", "OCO", "GAS", "HAD"],
    prev_teams=["mclaren", "haas"],
    extra_money=1.5,
    free_transfers=2,
    dollar_to_point=dollar_to_points
)

Budget: 98.8
Selected Players: ['ALB', 'BEA', 'HAD', 'LAW', 'PIA']
Selected Teams:   ['haas', 'mclaren']
Double Points:          PIA

Expected Points:       211.38
Expected Price Change: 2.06
Weighted Objective:    231.97
Total Transfers:       2
Extra Transfers:       0
Penalty Applied:       0

Drivers selected:
        ------     Price  E[Points]  E[PriceChange]  Price  Captain
Driver                                                             
BEA                  8.5     9.8750            0.48    8.5    False
HAD                  5.7     5.6875            0.13    5.7    False
PIA                 23.7    36.6875            0.30   23.7     True
LAW                  6.6     4.2500            0.14    6.6    False
ALB                 12.6    11.5000            0.11   12.6    False

Teams selected:
         ------     Price  E[Points]  E[PriceChange]  Price
Team                                                       
haas                 10.0    21.0625             0.6   10.0
mclaren    

In [46]:
def evaluate_team(drivers_selected, teams_selected, captain, extra_money=0, dollar_to_point=dollar_to_points):
    
    if (captain not in drivers_selected):
        raise ValueError("Captain is not in drivers selected")
    # === Points and price change breakdown ===
    total_points = sum(driver.loc[n, "E[Points]"] for n in drivers_selected) \
                 + sum(team.loc[n, "E[Points]"] for n in teams_selected) \
                 + (driver.loc[captain, "E[Points]"])

    total_price_change = (
        sum(driver.loc[n, "E[PriceChange]"] for n in drivers_selected) +
        sum(team.loc[n, "E[PriceChange]"] for n in teams_selected)
    )

    total_cost = sum(prices[n] for n in drivers_selected + teams_selected) + extra_money + total_price_change

    # === Objective Value ===
    weighted_objective = total_points + dollar_to_point * total_price_change

    # === Output ===
    print("Evaluating Given Selection:")
    print("Drivers Selected: ", sorted(drivers_selected))
    print("Teams Selected:   ", sorted(teams_selected))
    print("Double Points:         ", captain)

    print(f"\nExpected Points:       {total_points:.2f}")
    print(f"Expected Price Change: {total_price_change:.2f}")
    print(f"Weighted Objective:    {weighted_objective:.2f}")
    print(f"Expected Budget Afterwards: {total_cost}")

    sel_drivers_df = driver.loc[drivers_selected].assign(
        Price=lambda df: df.index.map(prices),
        Captain=lambda df: df.index == captain
    )
    sel_teams_df = team.loc[teams_selected].assign(Price=lambda df: df.index.map(prices))

    print("\nDrivers selected:")
    print(sel_drivers_df.iloc[:, 2:])
    print("\nTeams selected:")
    print(sel_teams_df.iloc[:, 2:])


In [47]:
evaluate_team(
    drivers_selected=["PIA", "ANT", "BEA", "HAD", "BOR"],
    teams_selected=["mclaren", "rb"],
    captain="PIA",
    extra_money=0,
    dollar_to_point=dollar_to_points
)

Evaluating Given Selection:
Drivers Selected:  ['ANT', 'BEA', 'BOR', 'HAD', 'PIA']
Teams Selected:    ['mclaren', 'rb']
Double Points:          PIA

Expected Points:       212.06
Expected Price Change: 1.84
Weighted Objective:    230.46
Expected Budget Afterwards: 105.14

Drivers selected:
        ------     Price  E[Points]  E[PriceChange]  Price  Captain
Driver                                                             
PIA                 23.7    36.6875            0.30   23.7     True
ANT                 19.6    17.6875           -0.07   19.6    False
BEA                  8.5     9.8750            0.48    8.5    False
HAD                  5.7     5.6875            0.13    5.7    False
BOR                  4.5     1.5625            0.15    4.5    False

Teams selected:
         ------     Price  E[Points]  E[PriceChange]  Price
Team                                                       
mclaren              31.5     85.625            0.30   31.5
rb                    9.8     18.250

In [48]:
evaluate_team(
    drivers_selected=["PIA", "ANT", "BEA", "HAD", "RUS"],
    teams_selected=["haas", "rb"],
    captain="PIA",
    extra_money=4.3,
    dollar_to_point=dollar_to_points
)

Evaluating Given Selection:
Drivers Selected:  ['ANT', 'BEA', 'HAD', 'PIA', 'RUS']
Teams Selected:    ['haas', 'rb']
Double Points:          PIA

Expected Points:       167.69
Expected Price Change: 2.08
Weighted Objective:    188.49
Expected Budget Afterwards: 105.78

Drivers selected:
        ------     Price  E[Points]  E[PriceChange]  Price  Captain
Driver                                                             
PIA                 23.7    36.6875            0.30   23.7     True
ANT                 19.6    17.6875           -0.07   19.6    False
BEA                  8.5     9.8750            0.48    8.5    False
HAD                  5.7     5.6875            0.13    5.7    False
RUS                 22.1    21.7500            0.09   22.1    False

Teams selected:
      ------     Price  E[Points]  E[PriceChange]  Price
Team                                                    
haas              10.0    21.0625            0.60   10.0
rb                 9.8    18.2500            0.5

In [49]:
evaluate_team(
    drivers_selected=["PIA", "ALB", "BEA", "HAD", "OCO"],
    teams_selected=["haas", "mclaren"],
    captain="PIA",
    extra_money=1.9,
    dollar_to_point=dollar_to_points
)

Evaluating Given Selection:
Drivers Selected:  ['ALB', 'BEA', 'HAD', 'OCO', 'PIA']
Teams Selected:    ['haas', 'mclaren']
Double Points:          PIA

Expected Points:       216.00
Expected Price Change: 2.14
Weighted Objective:    237.40
Expected Budget Afterwards: 105.14

Drivers selected:
        ------     Price  E[Points]  E[PriceChange]  Price  Captain
Driver                                                             
PIA                 23.7    36.6875            0.30   23.7     True
ALB                 12.6    11.5000            0.11   12.6    False
BEA                  8.5     9.8750            0.48    8.5    False
HAD                  5.7     5.6875            0.13    5.7    False
OCO                  9.1     8.8750            0.22    9.1    False

Teams selected:
         ------     Price  E[Points]  E[PriceChange]  Price
Team                                                       
haas                 10.0    21.0625             0.6   10.0
mclaren              31.5    85.62

In [50]:
evaluate_team(
    drivers_selected=["RUS", "BOR", "BEA", "HAD", "ALO"],
    teams_selected=["mercedes", "mclaren"],
    captain="RUS",
    extra_money=0,
    dollar_to_point=dollar_to_points
)

Evaluating Given Selection:
Drivers Selected:  ['ALO', 'BEA', 'BOR', 'HAD', 'RUS']
Teams Selected:    ['mclaren', 'mercedes']
Double Points:          RUS

Expected Points:       202.75
Expected Price Change: 1.39
Weighted Objective:    216.65
Expected Budget Afterwards: 104.09

Drivers selected:
        ------     Price  E[Points]  E[PriceChange]  Price  Captain
Driver                                                             
RUS                 22.1    21.7500            0.09   22.1     True
BOR                  4.5     1.5625            0.15    4.5    False
BEA                  8.5     9.8750            0.48    8.5    False
HAD                  5.7     5.6875            0.13    5.7    False
ALO                  6.2     3.0000           -0.06    6.2    False

Teams selected:
          ------     Price  E[Points]  E[PriceChange]  Price
Team                                                        
mercedes              24.2     53.500             0.3   24.2
mclaren               31.5 

# KARITOŞKO <3

In [51]:
optimize_team(
    prev_drivers=["PIA", "OCO", "BEA", "HAD", "ALB"],
    prev_teams=["rb", "mclaren"],
    extra_money=2.3,
    free_transfers=3,
    dollar_to_point=dollar_to_points
)

Budget: 103.2
Selected Players: ['BEA', 'BOR', 'DOO', 'HAD', 'PIA']
Selected Teams:   ['mclaren', 'mercedes']
Double Points:          PIA

Expected Points:       230.81
Expected Price Change: 1.37
Weighted Objective:    244.51
Total Transfers:       3
Extra Transfers:       0
Penalty Applied:       0

Drivers selected:
        ------     Price  E[Points]  E[PriceChange]  Price  Captain
Driver                                                             
BEA                  8.5     9.8750            0.48    8.5    False
HAD                  5.7     5.6875            0.13    5.7    False
BOR                  4.5     1.5625            0.15    4.5    False
PIA                 23.7    36.6875            0.30   23.7     True
DOO                  5.0     1.1875           -0.29    5.0    False

Teams selected:
          ------     Price  E[Points]  E[PriceChange]  Price
Team                                                        
mercedes              24.2     53.500             0.3   24.2
mcl

In [52]:
optimize_team(
    prev_drivers=["RUS", "ALO", "BEA", "HAD", "ALB"],
    prev_teams=["rb", "mclaren"],
    extra_money=1.1,
    free_transfers=2,
    dollar_to_point=dollar_to_points
)

Budget: 97.5
Selected Players: ['ALB', 'BEA', 'BOR', 'HAD', 'PIA']
Selected Teams:   ['mclaren', 'rb']
Double Points:          PIA

Expected Points:       205.88
Expected Price Change: 2.02
Weighted Objective:    226.07
Total Transfers:       2
Extra Transfers:       0
Penalty Applied:       0

Drivers selected:
        ------     Price  E[Points]  E[PriceChange]  Price  Captain
Driver                                                             
BEA                  8.5     9.8750            0.48    8.5    False
HAD                  5.7     5.6875            0.13    5.7    False
BOR                  4.5     1.5625            0.15    4.5    False
PIA                 23.7    36.6875            0.30   23.7     True
ALB                 12.6    11.5000            0.11   12.6    False

Teams selected:
         ------     Price  E[Points]  E[PriceChange]  Price
Team                                                       
rb                    9.8     18.250            0.55    9.8
mclaren      

In [53]:
optimize_team(
    prev_drivers=["NOR", "OCO", "BEA", "HAD", "STR"],
    prev_teams=["rb", "mclaren"],
    extra_money=1.5,
    free_transfers=2,
    dollar_to_point=dollar_to_points
)

Budget: 105.3
Selected Players: ['ALB', 'BEA', 'HAD', 'OCO', 'PIA']
Selected Teams:   ['mclaren', 'rb']
Double Points:          PIA

Expected Points:       213.19
Expected Price Change: 2.09
Weighted Objective:    234.09
Total Transfers:       2
Extra Transfers:       0
Penalty Applied:       0

Drivers selected:
        ------     Price  E[Points]  E[PriceChange]  Price  Captain
Driver                                                             
BEA                  8.5     9.8750            0.48    8.5    False
HAD                  5.7     5.6875            0.13    5.7    False
PIA                 23.7    36.6875            0.30   23.7     True
OCO                  9.1     8.8750            0.22    9.1    False
ALB                 12.6    11.5000            0.11   12.6    False

Teams selected:
         ------     Price  E[Points]  E[PriceChange]  Price
Team                                                       
rb                    9.8     18.250            0.55    9.8
mclaren     

### I think that following team is the best option for the first team

In [54]:
evaluate_team(["PIA", "ANT", "BOR", "HAD", "BEA"], ["mclaren", "haas"], "PIA")

Evaluating Given Selection:
Drivers Selected:  ['ANT', 'BEA', 'BOR', 'HAD', 'PIA']
Teams Selected:    ['haas', 'mclaren']
Double Points:          PIA

Expected Points:       214.88
Expected Price Change: 1.89
Weighted Objective:    233.78
Expected Budget Afterwards: 105.39

Drivers selected:
        ------     Price  E[Points]  E[PriceChange]  Price  Captain
Driver                                                             
PIA                 23.7    36.6875            0.30   23.7     True
ANT                 19.6    17.6875           -0.07   19.6    False
BOR                  4.5     1.5625            0.15    4.5    False
HAD                  5.7     5.6875            0.13    5.7    False
BEA                  8.5     9.8750            0.48    8.5    False

Teams selected:
         ------     Price  E[Points]  E[PriceChange]  Price
Team                                                       
mclaren              31.5    85.6250             0.3   31.5
haas                 10.0    21.06